In [13]:
import pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm import tqdm

import seaborn as sns
sns.set_style("whitegrid")

import urllib3

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
http = urllib3.PoolManager()

In [3]:
regions = {
    'Camden': 1946157246,
    'City of London': 1946157247,
    'Hackney': 1946157248,
    'Haringey': 1946157250,
    'Islington': 1946157251,
    'Kensington and Chelsea': 1946157252,
    'Lambeth': 1946157253,
    'Lewisham': 1946157254,
    'Newham': 1946157255,
    'Southwark': 1946157256,
    'Tower Hamlets': 1946157257,
    'Wandsworth': 1946157258,
    'Westminster': 1946157259,
    'Barking and Dagenham': 1946157260,
    'Barnet': 1946157261,
    'Bexley': 1946157262,
    'Brent': 1946157263,
    'Bromley': 1946157264,
    'Croydon': 1946157265,
    'Ealing': 1946157266,
    'Enfield': 1946157267,
    'Greenwich': 1946157268,
    'Harrow': 1946157269,
    'Havering': 1946157270,
    'Hillingdon': 1946157271,
    'Hounslow': 1946157272,
    'Kingston upon Thames': 1946157273,
    'Merton': 1946157274,
    'Redbridge': 1946157275,
    'Richmond upon Thames': 1946157276,
    'Sutton': 1946157277,
    'Waltham Forest': 1946157278,
    'Hammersmith & Fulham': 1946157249,
}

In [4]:
def compute_stats(freq_list_orig, freq_list_synth):
    """
    Compute different statistics (MAE, RMSE, SMRSE, R^2, and Pearson's correlation) on two frequency lists.

    Parameters
    ----------
    freq_list_orig: numpy.ndarray
        Frequency list for the original data
    freq_list_synth: numpy.ndarray
        Frequency list for the synthetic data

    Returns
    -------
    stat: dict
        Dictionary of the stats between the two lists
    """

    freq_list_orig, freq_list_synth = np.array(freq_list_orig), np.array(freq_list_synth)
    corr_mat = np.corrcoef(freq_list_orig, freq_list_synth)
    corr = corr_mat[0, 1]
    if np.isnan(corr): corr = 0.0
    # MAE
    mae = np.absolute(freq_list_orig - freq_list_synth).mean()
    # RMSE
    rmse = np.linalg.norm(freq_list_orig - freq_list_synth) / np.sqrt(len(freq_list_orig))
    # SRMSE
    freq_list_orig_avg = freq_list_orig.mean()
    srmse = rmse / freq_list_orig_avg
    # r-square
    u = np.sum((freq_list_synth - freq_list_orig) ** 2)
    v = np.sum((freq_list_orig - freq_list_orig_avg) ** 2)
    r2 = 1.0 - u / v
    stat = {'mae': mae, 'rmse': rmse, 'r2': r2, 'srmse': srmse, 'corr': corr}

    return stat

# Testing against nomis

Here, we want to test the generated data against the data recorded in the nomis database.

We test on the following variables:
* Distance travelled to work (QS702EW - NM_153_1)
* Method of travel to work (QS701EW - NM_568_1)
* Number of people in household (QS406EW - NM_538_1)

In [14]:
lpmc = pd.read_csv('../../data/LPMC/trips.csv')
lpmc.hh_borough.value_counts()

Croydon                   808
Lambeth                   758
Wandsworth                748
Bromley                   695
Barnet                    669
Ealing                    654
Brent                     640
Lewisham                  605
Southwark                 590
Haringey                  565
Hillingdon                531
Enfield                   529
Hammersmith & Fulham      527
Greenwich                 521
Waltham Forest            515
Merton                    505
Camden                    499
Redbridge                 490
Hackney                   476
Richmond upon Thames      469
Newham                    466
Hounslow                  464
Islington                 450
Tower Hamlets             445
Kensington and Chelsea    438
Havering                  428
Harrow                    428
Sutton                    417
Bexley                    413
Kingston upon Thames      358
Barking and Dagenham      316
Westminster               307
City of London            180
Spelthorne

## Number of people in household

In [244]:
lpmc = pd.read_csv('../../data/LPMC/trips.csv')
stats_str = ['mae', 'rmse', 'r2', 'srmse', 'corr']

In [241]:
res = {
    'synth': {},
    'lpmc': {},
    'spec': {}
}

for i in res:
    for s in stats_str:
        res[i][s] = []

    res[i]['count'] = []

for r in tqdm(regions):

    # Get the data
    url = 'https://www.nomisweb.co.uk/api/v01/dataset/NM_538_1.data.csv?geography={}&rows=cell&cols=rural_urban&measures=20100'.format(regions[r])
    response = http.request('GET', url)

    with open('tmp.txt', 'w') as f:
        f.write(response.data.decode('utf-8'))

    df = pd.read_csv('tmp.txt')

    nomis = list(df['GROUP 1: OBS_VALUE'].iloc[1:])

    tmp = list(lpmc['hh_people'].value_counts().sort_index(ascending=True))
    tmp[7] = sum(tmp[7:])
    orig = tmp[:8]

    synth = pd.read_csv('../../data/synthetic/LPMC_cond_{}.csv'.format(r))
    tmp = list(synth['hh_people'].value_counts().sort_index(ascending=True))
    tmp[len(nomis)-1] = sum(tmp[len(nomis)-1:])
    synth = tmp[:len(nomis)]

    spec = lpmc[lpmc['hh_borough'] == r]
    tmp = spec['hh_people'].value_counts().sort_index(ascending=True)
    spec = np.zeros(len(nomis))
    for i in tmp.index:
        if i-1 >= len(spec):
            spec[len(spec)-1] += tmp[i]
        else:
            spec[i-1] = tmp[i]

    res['lpmc']['count'].append(sum(orig)/sum(nomis)*100)
    res['spec']['count'].append(sum(spec)/sum(nomis)*100)
    res['synth']['count'].append(sum(synth)/sum(nomis)*100)

    nomis = np.array(nomis)/sum(nomis)
    synth = np.array(synth)/sum(synth)
    spec = np.array(spec)/sum(spec)
    orig = np.array(orig)/sum(orig)

    res1 = compute_stats(nomis, orig)
    res2 = compute_stats(nomis, spec)
    res3 = compute_stats(nomis, synth)
    for s in stats_str:
        res['lpmc'][s].append(res1[s])
        res['spec'][s].append(res2[s])
        res['synth'][s].append(res3[s])

100%|██████████| 49/49 [00:51<00:00,  1.06s/it]


In [245]:
dct = {}

stats_str.append('count')

for s in stats_str:
    dct[s] = {
        'lpmc': np.array(res['lpmc'][s]).mean(),
        'spec': np.array(res['spec'][s]).mean(),
        'synth': np.array(res['synth'][s]).mean()
    }

df = pd.DataFrame(dct, index=['lpmc', 'spec', 'synth'])

In [246]:
df

,mae,rmse,r2,srmse,corr,count
lpmc,0.016119,0.023466,0.948883,0.187727,0.979826,34.662938
spec,0.030330,0.043993,0.804613,0.351944,0.914905,0.454787
synth,0.026882,0.037038,0.899499,0.296302,0.956574,247.527703


## Distance travelled to work

In [258]:
lpmc = pd.read_csv('../../data/LPMC/trips.csv')
stats_str = ['mae', 'rmse', 'r2', 'srmse', 'corr']

bins = [-np.inf, 2000, 5000, 10000, 20000, 30000, 40000, 60000, np.inf]
bin_labels = ['<2km', '2-5km', '5-10km', '10-20km', '20-30km', '30-40km', '40-60km', '>60km']

lpmc['distance'] = pd.cut(lpmc['distance'], bins=bins, labels=bin_labels)
lpmc = lpmc[lpmc['purpose'] == 'HBW']
lpmc = lpmc[((lpmc['age'] >= 16) & (lpmc['age'] <= 74))]

In [259]:
res = {
    'synth': {},
    'lpmc': {},
    'spec': {}
}

for i in res:
    for s in stats_str:
        res[i][s] = []
    res[i]['count'] = []

for r in tqdm(regions):

    # Get the data
    url = 'https://www.nomisweb.co.uk/api/v01/dataset/NM_153_1.data.csv?geography={}&rows=cell&cols=rural_urban&measures=20100'.format(regions[r])
    response = http.request('GET', url)

    with open('tmp.txt', 'w') as f:
        f.write(response.data.decode('utf-8'))

    df = pd.read_csv('tmp.txt')
    df = df.sort_values('CELL')
    df.index = df['CELL']

    nomis = list(df['GROUP 1: OBS_VALUE'].iloc[1:9])

    orig = list(lpmc['distance'].value_counts())

    synth = pd.read_csv('../../data/synthetic/LPMC_cond_{}.csv'.format(r))
    synth['distance'] = pd.cut(synth['distance'], bins=bins, labels=bin_labels)

    synth = synth[synth['purpose'] == 'HBW']
    synth = synth[((synth['age'] >= 16) & (synth['age'] <= 74))]
    tmp = synth['distance'].value_counts()

    synth = np.zeros(len(bin_labels))
    for i, o in enumerate(bin_labels):
        if o in tmp.index:
            synth[i] = tmp[o]

    spec = lpmc[lpmc['hh_borough'] == r]
    tmp = spec['distance'].value_counts()

    spec = np.zeros(len(bin_labels))
    for i, o in enumerate(bin_labels):
        if o in tmp.index:
            spec[i] = tmp[o]

    res['lpmc']['count'].append(sum(orig)/sum(nomis)*100)
    res['spec']['count'].append(sum(spec)/sum(nomis)*100)
    res['synth']['count'].append(sum(synth)/sum(nomis)*100)

    nomis = np.array(nomis)/sum(nomis)
    synth = np.array(synth)/sum(synth)
    orig = np.array(orig)/sum(orig)

    if sum(spec) > 0:
        spec = np.array(spec)/sum(spec)
        res2 = compute_stats(nomis, spec)

    res1 = compute_stats(nomis, orig)
    res3 = compute_stats(nomis, synth)
    for s in stats_str:
        res['lpmc'][s].append(res1[s])
        if sum(spec) > 0:
            res['spec'][s].append(res2[s])
        res['synth'][s].append(res3[s])

100%|██████████| 49/49 [00:55<00:00,  1.13s/it]


In [260]:
dct = {}

stats_str.append('count')

for s in stats_str:
    dct[s] = {
        'lpmc': np.array(res['lpmc'][s]).mean(),
        'spec': np.array(res['spec'][s]).mean(),
        'synth': np.array(res['synth'][s]).mean()
    }

df = pd.DataFrame(dct, index=['lpmc', 'spec', 'synth'])

In [261]:
df

,mae,rmse,r2,srmse,corr,count
lpmc,0.068422,0.092779,0.090982,0.742228,0.700115,7.468569
spec,0.053864,0.073514,-0.791099,0.588111,0.858448,0.094354
synth,0.051299,0.071228,0.377545,0.569822,0.829818,42.060229


## Method of travel to work

In [262]:
lpmc = pd.read_csv('../../data/LPMC/trips.csv')
lpmc = lpmc[lpmc['purpose'] == 'HBW']
lpmc = lpmc[((lpmc['age'] >= 16) & (lpmc['age'] <= 74))]

order = ['pt', 'drive', 'walk', 'cycle']

stats_str = ['mae', 'rmse', 'r2', 'srmse', 'corr']

In [263]:
res = {
    'synth': {},
    'lpmc': {},
    'spec': {}
}

for i in res:
    for s in stats_str:
        res[i][s] = []
    res[i]['count'] = []

for r in tqdm(regions):

    # Get the data
    url = 'https://www.nomisweb.co.uk/api/v01/dataset/NM_568_1.data.csv?geography={}&rows=cell&cols=rural_urban&measures=20100'.format(regions[r])
    response = http.request('GET', url)

    with open('tmp.txt', 'w') as f:
        f.write(response.data.decode('utf-8'))

    df = pd.read_csv('tmp.txt')
    df = df.sort_values('CELL')
    df.index = df['CELL']

    values = df['GROUP 1: OBS_VALUE']
    drive = np.sum(values[6:9])
    pt = np.sum(values[2:6])
    walk = values[10]
    cycle = values[9]

    nomis = [pt, drive, walk, cycle]

    orig = list(lpmc['travel_mode'].value_counts())

    synth = pd.read_csv('../../data/synthetic/LPMC_cond_{}.csv'.format(r))

    synth = synth[synth['purpose'] == 'HBW']
    synth = synth[((synth['age'] >= 16) & (synth['age'] <= 74))]
    tmp = synth['travel_mode'].value_counts()

    synth = np.zeros(len(order))
    for i, o in enumerate(order):
        synth[i] = tmp[o]

    spec = lpmc[lpmc['hh_borough'] == r]
    tmp = spec['travel_mode'].value_counts()

    spec = np.zeros(len(order))
    for i, o in enumerate(order):
        if o in tmp.index:
            spec[i] = tmp[o]

    res['lpmc']['count'].append(sum(orig)/sum(nomis)*100)
    res['spec']['count'].append(sum(spec)/sum(nomis)*100)
    res['synth']['count'].append(sum(synth)/sum(nomis)*100)

    nomis = np.array(nomis)/sum(nomis)
    synth = np.array(synth)/sum(synth)
    orig = np.array(orig)/sum(orig)

    if sum(spec) > 0:
        spec = np.array(spec)/sum(spec)
        res2 = compute_stats(nomis, spec)

    res1 = compute_stats(nomis, orig)
    res3 = compute_stats(nomis, synth)
    for s in stats_str:
        res['lpmc'][s].append(res1[s])
        if sum(spec) > 0:
            res['spec'][s].append(res2[s])
        res['synth'][s].append(res3[s])

100%|██████████| 49/49 [01:04<00:00,  1.32s/it]


In [266]:
dct = {}

stats_str.append('count')

for s in stats_str:
    dct[s] = {
        'lpmc': np.array(res['lpmc'][s]).mean(),
        'spec': np.array(res['spec'][s]).mean(),
        'synth': np.array(res['synth'][s]).mean()
    }

df = pd.DataFrame(dct, index=['lpmc', 'spec', 'synth'])

In [267]:
df

,mae,rmse,r2,srmse,corr,count
lpmc,0.119683,0.154335,0.454048,0.617340,0.693390,6.410999
spec,0.076303,0.093179,0.712300,0.372714,0.888302,0.080608
synth,0.127243,0.161286,0.397791,0.645142,0.655271,35.629151


## Number of cars in the household

In [283]:
lpmc = pd.read_csv('../../data/LPMC/trips.csv')

stats_str = ['mae', 'rmse', 'r2', 'srmse', 'corr']

In [284]:
res = {
    'synth': {},
    'lpmc': {},
    'spec': {}
}

for i in res:
    for s in stats_str:
        res[i][s] = []

    res[i]['count'] = []

for r in tqdm(regions):

    # Get the data
    url = 'https://www.nomisweb.co.uk/api/v01/dataset/NM_621_1.data.csv?geography={}&rows=cell&cols=rural_urban&measures=20100'.format(regions[r])
    response = http.request('GET', url)

    with open('tmp.txt', 'w') as f:
        f.write(response.data.decode('utf-8'))

    df = pd.read_csv('tmp.txt')

    nomis = list(df['GROUP 1: OBS_VALUE'].iloc[1:6])

    tmp = list(lpmc['hh_vehicles'].value_counts().sort_index(ascending=True))
    tmp[4] = sum(tmp[4:])
    orig = tmp[:5]

    synth = pd.read_csv('../../data/synthetic/LPMC_cond_{}.csv'.format(r))
    tmp = list(synth['hh_vehicles'].value_counts().sort_index(ascending=True))
    tmp[len(nomis)-1] = sum(tmp[len(nomis)-1:])
    synth = tmp[:len(nomis)]

    spec = lpmc[lpmc['hh_borough'] == r]
    tmp = spec['hh_vehicles'].value_counts().sort_index(ascending=True)
    spec = np.zeros(len(nomis))
    for i in tmp.index:
        if i-1 >= len(spec):
            spec[len(spec)-1] += tmp[i]
        else:
            spec[i-1] = tmp[i]

    res['lpmc']['count'].append(sum(orig)/sum(nomis)*100)
    res['spec']['count'].append(sum(spec)/sum(nomis)*100)
    res['synth']['count'].append(sum(synth)/sum(nomis)*100)

    nomis = np.array(nomis)/sum(nomis)
    synth = np.array(synth)/sum(synth)
    spec = np.array(spec)/sum(spec)
    orig = np.array(orig)/sum(orig)

    res1 = compute_stats(nomis, orig)
    res2 = compute_stats(nomis, spec)
    res3 = compute_stats(nomis, synth)
    for s in stats_str:
        res['lpmc'][s].append(res1[s])
        res['spec'][s].append(res2[s])
        res['synth'][s].append(res3[s])

100%|██████████| 49/49 [00:51<00:00,  1.05s/it]


In [285]:
dct = {}

stats_str.append('count')

for s in stats_str:
    dct[s] = {
        'lpmc': np.array(res['lpmc'][s]).mean(),
        'spec': np.array(res['spec'][s]).mean(),
        'synth': np.array(res['synth'][s]).mean()
    }

df = pd.DataFrame(dct, index=['lpmc', 'spec', 'synth'])

In [286]:
df

,mae,rmse,r2,srmse,corr,count
lpmc,0.069463,0.091656,0.625632,0.458279,0.845011,34.662938
spec,0.162456,0.212049,-0.514296,1.060245,0.444313,0.410636
synth,0.066459,0.087525,0.662086,0.437624,0.858153,247.527703
